# GCN
This notebook establishes a training pipeline for our Event Embedding model.

In [ ]:
!git clone https://github.com/dbusbridge/gcn_tutorial

Cloning into 'gcn_tutorial'...
remote: Enumerating objects: 69, done.
remote: Total 69 (delta 0), reused 0 (delta 0), pack-reused 69
Unpacking objects: 100% (69/69), done.


In [ ]:
!mv gcn_tutorial/* .

In [ ]:
!pip install tensorflow==1.15.2

     |████████████████████████████████| 110.5MB 47kB/s 
     |████████████████████████████████| 512kB 36.1MB/s 
     |████████████████████████████████| 3.8MB 36.8MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=5640b5ceb26cb8d2ace45e452b6d76dedc3156bc17e3992ba95e31575e57ff7d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninsta

In [ ]:
!gdown --id 1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0
!tar -xzvf datasets.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0
To: /content/datasets.tar.gz
22.7MB [00:00, 86.2MB/s]
datasets_runs/
datasets_runs/run_1_google_news_5w1h_graph_hin.nx
datasets_runs/run_6_40er_5w1h_graph_hin.nx
datasets_runs/run_4_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_5_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_google_news_5w1h_graph_hin.nx
datasets_runs/run_5_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_2_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_7_40er_5w1h_graph_hin.nx
datasets_runs/run_9_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_8_google_news_5w1h_graph_hin.nx
datasets_runs/run_10_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_2_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_8_40er_5w1h_graph_hin.nx
datasets_runs/run_6_bbc_5w1h_graph_hin.nx
datasets_runs/run_4_google_news_5w1h_graph_hin

In [ ]:
import networkx as nx
import numpy as np
import time

import scipy.sparse

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import tensorflow as tf

import layers.graph as lg
import utils.sparse as us

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
from os import listdir
from os.path import isfile, join
path_datasets = 'datasets_runs/'
network_files = [f for f in listdir(path_datasets) if isfile(join(path_datasets, f))]
print(network_files)

['run_1_google_news_5w1h_graph_hin.nx', 'run_6_40er_5w1h_graph_hin.nx', 'run_4_bbc_5w1h_graph_hin.nx', 'run_8_gold_standard_5w1h_graph_hin.nx', 'run_5_bbc_5w1h_graph_hin.nx', 'run_9_google_news_5w1h_graph_hin.nx', 'run_5_gold_standard_5w1h_graph_hin.nx', 'run_2_bbc_5w1h_graph_hin.nx', 'run_9_news_cluster_5w1h_graph_hin.nx', 'run_7_40er_5w1h_graph_hin.nx', 'run_9_gold_standard_5w1h_graph_hin.nx', 'run_8_google_news_5w1h_graph_hin.nx', 'run_10_bbc_5w1h_graph_hin.nx', 'run_8_news_cluster_5w1h_graph_hin.nx', 'run_2_news_cluster_5w1h_graph_hin.nx', 'run_8_40er_5w1h_graph_hin.nx', 'run_6_bbc_5w1h_graph_hin.nx', 'run_4_google_news_5w1h_graph_hin.nx', 'run_2_google_news_5w1h_graph_hin.nx', 'run_7_gold_standard_5w1h_graph_hin.nx', 'run_4_gold_standard_5w1h_graph_hin.nx', 'run_5_40er_5w1h_graph_hin.nx', 'run_3_gold_standard_5w1h_graph_hin.nx', 'run_4_40er_5w1h_graph_hin.nx', 'run_5_google_news_5w1h_graph_hin.nx', 'run_10_news_cluster_5w1h_graph_hin.nx', 'run_10_40er_5w1h_graph_hin.nx', 'run_9_40

In [ ]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [ ]:
from tqdm.notebook import tqdm

experimental_results = []

for network in tqdm(network_files):

  print('Reading',network)
  G = nx.read_gpickle(path_datasets+network)

  node_list = []
  for node in G.nodes():
    node_list.append(node)

  label_codes = {}
  for node in node_list:
    if 'label' in G.nodes[node]:
      label = G.nodes[node]['label']
      if label not in label_codes: label_codes[label] = len(label_codes)
      G.nodes[node]['membership'] = label_codes[label]
    else: G.nodes[node]['membership'] = -1

  adj = nx.adj_matrix(G,nodelist=node_list)

  # Get important parameters of adjacency matrix
  n_nodes = adj.shape[0]

  # Some preprocessing
  adj_tilde = adj + np.identity(n=adj.shape[0])
  d_tilde_diag = np.squeeze(np.sum(np.array(adj_tilde), axis=1))
  d_tilde_inv_sqrt_diag = np.power(d_tilde_diag, -1/2)
  d_tilde_inv_sqrt = np.diag(d_tilde_inv_sqrt_diag)
  adj_norm = np.dot(np.dot(d_tilde_inv_sqrt, adj_tilde), d_tilde_inv_sqrt)
  adj_norm_tuple = us.sparse_to_tuple(scipy.sparse.coo_matrix(adj_norm))

  # Features are just the identity matrix
  feat_x = np.identity(n=adj.shape[0])
  feat_x_tuple = us.sparse_to_tuple(scipy.sparse.coo_matrix(feat_x))

  # Preparing train data
  memberships = [m for m in nx.get_node_attributes(G, 'membership').values()]
  nb_classes = len(set(memberships))
  targets = np.array([memberships], dtype=np.int32).reshape(-1)
  one_hot_targets = np.eye(nb_classes)[targets]

  labels_to_keep = []

  counter = 0
  for node in node_list:
    if 'train' in G.nodes[node]:
      labels_to_keep.append(counter)
    counter += 1

  y_train = np.zeros(shape=one_hot_targets.shape,
                    dtype=np.float32)

  train_mask = np.zeros(shape=(n_nodes,), dtype=np.bool)

  for l in labels_to_keep:
      y_train[l, :] = one_hot_targets[l, :]
      train_mask[l] = True

  # TensorFlow placeholders
  ph = {
      'adj_norm': tf.sparse_placeholder(tf.float32, name="adj_mat"),
      'x': tf.sparse_placeholder(tf.float32, name="x"),
      'labels': tf.placeholder(tf.float32, shape=(n_nodes, nb_classes)),
      'mask': tf.placeholder(tf.int32)}

  l_sizes = [128, 128, 64, nb_classes]

  o_fc1 = lg.GraphConvLayer(
      input_dim=feat_x.shape[-1],
      output_dim=l_sizes[0],
      name='fc1_'+network,
      activation=tf.nn.tanh)(adj_norm=ph['adj_norm'], x=ph['x'], sparse=True)

  o_fc2 = lg.GraphConvLayer(
      input_dim=l_sizes[0],
      output_dim=l_sizes[1],
      name='fc2_'+network,
      activation=tf.nn.tanh)(adj_norm=ph['adj_norm'], x=o_fc1)

  o_fc3 = lg.GraphConvLayer(
      input_dim=l_sizes[1],
      output_dim=l_sizes[2],
      name='fc3_'+network,
      activation=tf.nn.tanh)(adj_norm=ph['adj_norm'], x=o_fc2)

  o_fc4 = lg.GraphConvLayer(
      input_dim=l_sizes[2],
      output_dim=l_sizes[3],
      name='fc4_'+network,
      activation=tf.identity)(adj_norm=ph['adj_norm'], x=o_fc3)


  with tf.name_scope('optimizer'):
      loss = masked_softmax_cross_entropy(preds=o_fc4, labels=ph['labels'], mask=ph['mask'])
      accuracy = masked_accuracy(preds=o_fc4, labels=ph['labels'], mask=ph['mask'])
      optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
      opt_op = optimizer.minimize(loss)

  feed_dict_train = {ph['adj_norm']: adj_norm_tuple,
                    ph['x']: feat_x_tuple,
                    ph['labels']: y_train,
                    ph['mask']: train_mask}

  sess = tf.Session()
  sess.run(tf.global_variables_initializer())

  epochs = 20
  save_every = 50

  t = time.time()
  outputs = {}
  # Train model
  for epoch in range(epochs):
      _, train_loss, train_acc = sess.run(
          (opt_op, loss, accuracy), feed_dict=feed_dict_train)

      if True:
          val_loss, val_acc = sess.run((loss, accuracy), feed_dict=feed_dict_train)

          # # Print results
          # #print("Epoch:", '%04d' % (epoch + 1),
          #       "train_loss=", "{:.5f}".format(train_loss),
          #       "time=", "{:.5f}".format(time.time() - t))

          feed_dict_output = {ph['adj_norm']: adj_norm_tuple,
                              ph['x']: feat_x_tuple}

          embeddings = sess.run(o_fc3, feed_dict=feed_dict_output)
          preds = sess.run(o_fc4, feed_dict=feed_dict_output)
          outputs['embeddings_'+str(epoch)] = embeddings
          outputs['predictions_'+str(epoch)] = preds

  predictions = np.argmax(preds,axis=1)
  y_pred = []
  y_true = []
  counter = 0
  for node in node_list:
    if 'test' in G.nodes[node]:
      y_true.append( label_codes[G.nodes[node]['label']] )
      y_pred.append( predictions[counter] )

    counter += 1

  f1_macro = f1_score(y_true, y_pred, average='macro')
  acc = accuracy_score(y_true, y_pred)

  print('--->' ,network,'f1_macro',f1_macro,'acc',acc)
  experimental_results.append((network,'f1_macro',f1_macro,'acc',acc,y_true,y_pred))

Reading run_1_google_news_5w1h_graph_hin.nx



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

---> run_1_google_news_5w1h_graph_hin.nx f1_macro 0.6072974644403216 acc 0.6
Reading run_6_40er_5w1h_graph_hin.nx
---> run_6_40er_5w1h_graph_hin.nx f1_macro 0.7509700176366843 acc 0.8125
Reading run_4_bbc_5w1h_graph_hin.nx
---> run_4_bbc_5w1h_graph_hin.nx f1_macro 0.4440781440781441 acc 0.45454545454545453
Reading run_8_gold_standard_5w1h_graph_hin.nx
---> run_8_gold_standard_5w1h_graph_hin.nx f1_macro 0.505508331012717 acc 0.618421052631579
Reading run_5_bbc_5w1h_graph_hin.nx
---> run_5_bbc_5w1h_graph_hin.nx f1_macro 0.4535652987985645 acc 0.45454545454545453
Reading run_9_google_news_5w1h_graph_hin.nx
---> run_9_google_news_5w1h_graph_hin.nx f1_macro 0.6757369614512471 acc 0.72
Reading run_5_gold_standard_5w1h_graph_hin.nx
---> run_5_gold_standard_5w

In [ ]:
import pandas as pd

df_results = pd.DataFrame(experimental_results)

In [ ]:
df_results.to_excel('gcn_results_r1.xls' )